<a href="https://colab.research.google.com/github/Prattoypaul/deeplearning/blob/machinel/Skin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os 
import cv2
import seaborn as sb
import  tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
!nvidia-smi

In [ ]:
pic_size=48

In [ ]:
datadir='/content/drive/MyDrive/Colab Notebooks/A==4 chatbot/Train'

In [ ]:
classes=['actinic keratosis','basal cell carcinoma','dermatofibroma','nevus','pigmented benign keratosis','seborrheic keratosis','squamous cell carcinoma','vascular lesion']

In [ ]:
dataset=keras.preprocessing.image_dataset_from_directory(
    datadir,seed=123,
    shuffle=True,image_size=(256,256),batch_size=16)

In [ ]:
for img ,lab in dataset.take(1):
  print(img.shape)
  print(lab.numpy())

In [ ]:
for img,label in dataset.take(1):
  print(img.shape)
  print(label.numpy())

In [ ]:
len(dataset)

In [ ]:
len(dataset)*0.8

In [ ]:
train_ds=dataset.take(118)


In [ ]:
test=dataset.skip(118)
len(test)

In [ ]:
val_ds=test.take(15)
test_ds=test.skip(15)

In [ ]:
train_ds=train_ds.cache().shuffle(100000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(100000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
plt.figure(figsize=(15,15))
for img_b,lab_b in dataset.take(1):
  for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(img_b[i].numpy().astype('uint8'))
    plt.title(classes[lab_b[i]])
    plt.axis('off')

In [ ]:
resize_rescale=keras.Sequential([
                                 keras.layers.experimental.preprocessing.Resizing(256,256),
                                 keras.layers.experimental.preprocessing.Rescaling(1/255)
])

In [ ]:
data_augment=keras.Sequential([
                               keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
                               keras.layers.experimental.preprocessing.RandomRotation(0.3)
])

In [ ]:
train_ds=train_ds.map(
    lambda x,y:(data_augment(x,training=True),y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
input_shape=(16,256,256,3)

In [ ]:
model=keras.Sequential([
                        resize_rescale,
                        keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Conv2D(64,(3,3),activation='relu'),
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Conv2D(64,(3,3),activation='relu'),
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Conv2D(64,(3,3),activation='relu'),
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Conv2D(64,(3,3),activation='relu'),
                        keras.layers.MaxPooling2D((2,2)),
                       
                        keras.layers.MaxPooling2D((2,2)),
                        keras.layers.Flatten(),
                        keras.layers.Dense(64,activation='relu'),
                        keras.layers.Dense(32,activation='relu'),
                        keras.layers.Dense(9,activation='softmax')


])


In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics='accuracy'
)

In [ ]:
model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
ht=model.fit(train_ds,batch_size=16,verbose=1,validation_data=val_ds,epochs=45)

In [ ]:
model.evaluate(test_ds)

In [ ]:
ht.params

In [ ]:
type(ht.history['loss'])

In [ ]:
len(ht.history['loss'])

In [ ]:
ht.history.keys()

In [ ]:
loss=ht.history['loss']
Acc=ht.history['accuracy']
val_loss=ht.history['val_loss']
val_acc=ht.history['val_accuracy']

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
plt.plot(range(45),Acc,label='Train_accuracy')
plt.plot(range(45),val_acc,label='Validation_Acc')
plt.legend(loc='upper right')
plt.title('Accuracy Compare')
plt.subplot(1,2,2)
plt.plot(range(45),loss,label='Train_loss')
plt.plot(range(45),val_loss,label='Validation_loss')
plt.legend(loc='lower right')
plt.title('loss Compare')
plt.show()

In [ ]:
for img , lab in test_ds.take(1):
  first_img=img[0].numpy().astype('uint8')
  first_lab=lab[0].numpy()
  plt.imshow(first_img)
  print('actual_label:',classes[first_lab])
  batch_prediction=model.predict(img)
  print('predict_label:',classes[np.argmax(batch_prediction[0])])




In [ ]:
from IPython.core.display import Image
def predict(model,img):
  img_array=keras.preprocessing.image.img_to_array(images[i].numpy())
  img_array=tf.expand_dims(img_array,0)

  img_predict=model.predict(img_array)
  predict_class=classes[np.argmax(img_predict[0])]
  confidence=round(100*(np.max(img_predict[0])),2)
  return predict_class,confidence
  

In [ ]:
plt.figure(figsize=(20,20))
for images,lab in test_ds.take(1):
  for i in range(9):
    ax=plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    predicted_class,confidence=predict(model,images[i].numpy())
    actual_lab=classes[lab[i]]
    plt.title(f"Actual label:{actual_lab}\n Predict:{predicted_class}\n Confidence:{confidence}")
    plt.axis('off')